In [534]:
import numpy as np
import pandas as pd
import sklearn as sk
#hyper-parameters
INPUT_LAYER_SIZE=784
OUTPUT_LAYER_SIZE=10
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [535]:
train=pd.read_csv('./data/train.csv')
val=pd.read_csv('./data/val.csv')
test=pd.read_csv('./data/test.csv')

In [538]:
training_data_class =train.iloc[:,-1].copy().as_matrix()
training_data=sk.preprocessing.normalize(train.iloc[:,1:785].copy().as_matrix())

In [539]:
validating_data_class =val.iloc[:,-1].copy().as_matrix()
validating_data=sk.preprocessing.normalize(val.iloc[:,1:785].copy().as_matrix())

clf = LinearDiscriminantAnalysis()
clf.fit(training_data, training_data_class)


training_data=clf.transform(training_data)
validating_data=clf.transform(validating_data)

In [540]:
print training_data.shape

(55000, 784)


In [541]:
def create_output(class_labels):
    output=np.zeros((len(class_labels),10))
    for i in range(len(class_labels)):
        output[i,class_labels[i]]=1
    return output
def sigmoid(value):
    return 1/(1+np.exp(-value))
def tanh(value):
    return np.tanh(value)
def sigmoid_dif(value):
    return sigmoid(value)*(1-sigmoid(value))
def tanh_dif(value):
    return 1-tanh(value)*tanh(value)
def softmax(vector):
    num= np.exp(vector)
    return num/np.sum(num)
def paraCopy(w,b,multiplier=1):
    return [i*multiplier for i in w],[i*multiplier for i in b]
def paraAdd(A,B):
    a,b=A
    c,d=B
    return [i+j for i,j in zip(a,c)],[i+j for i,j in zip(b,d)]

In [542]:
def shaper(lst):
    for i in lst:
        print 'shape -',i.shape
def loss(list1,list2):
    return -np.sum([j*np.log2(i) for i,j in zip(list1,list2)])
def dumpModel(model):
    kp=0
    for i,j in zip(model.weights,model.biases):
        np.save('./temp/weights_'+str(kp),i)
        np.save('./temp/biases_'+str(kp),j)
        kp+=1


In [550]:
class NN_Model:
    def __init__(self,lr=0.01,momentum=0,hidden_layers=(100,),activation='sigmoid',loss='sq',
                 opt='gd',batch_size=1,anneal=False,save_dir='./temp/',expt_dir='./temp/',max_iter=50):
        self.lr=lr
        self.momentum=momentum
        self.layers=(INPUT_LAYER_SIZE,)+hidden_layers+(OUTPUT_LAYER_SIZE,)
        if activation=='sigmoid':
            self.activation=sigmoid
        elif activation=='tanh':
            self.activation=tanh
        else:
            print 'Error : activation function not found'
                
        
        self.loss=loss
        self.opt=opt
        self.batch_size=batch_size
        self.anneal=anneal
        self.save_dir=save_dir
        self.expt_dir=expt_dir
        self.num_layers=len(self.layers)
        self.max_iter=max_iter
    
    def __forward_propagation(self):
        h_set=[]
        a_set=[]
        h=self.input_data[TRAINER]
        h_set.append(h) #experimental
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
            a_set.append(a)
            h_set.append(h)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        a_set.append(a)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return h_set,a_set,y
    
    def __forward_propagation_test(self,h):
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return y
    
    
    def __back_propagation(self,h,a,y):
        L=self.num_layers-2
        dWeights=[]
        dBiases=[]
        
        daL_loss=-(self.output_data[TRAINER]-y) # for cross-entropy loss function
        dA_loss=daL_loss
        for k in range(L,-1,-1):
            #print 'k=',k
            dW_loss=np.outer(dA_loss,h[k]).T
            dB_loss=dA_loss
            
            if k!=0:
                dH_loss=np.matmul(self.weights[k].T,dA_loss)
            
                dA_loss=dH_loss*sigmoid_dif(a[k-1])
            
            
            dWeights=[dW_loss.T]+dWeights
            dBiases=[dB_loss.T]+dBiases
            #print dA_loss.shape
            #return _,_
            
            
        return dWeights,dBiases
    
    def __update(self,dWeights,dBiases):
        for i in range(len(self.weights)):
            self.weights[i]=self.weights[i]-dWeights[i]
            self.biases[i]=self.biases[i]-dBiases[i]

    
    
    
    def __train(self):
        global TRAINER
        for i in range(self.max_iter):
            
            accuracy=0
            if self.batch_size==1:
                self.batch_size=len(training_data)
            
            sets=len(training_data)/self.batch_size # number of batches
                
            oldDWeights,oldDBiases=paraCopy(self.weights,self.biases,0) # for momentum
            for j in range(sets):
                #print 'SET=',j
                los=0
                dWeights,dBiases=paraCopy(self.weights,self.biases,0)
                for TRAINER in range(j*self.batch_size,(j+1)*self.batch_size):
                    #print 'TRAINER=',TRAINER
                    h,a,y=self.__forward_propagation()

                    
                    dWeights,dBiases=paraAdd(self.__back_propagation(h,a,y),
                                         paraCopy(dWeights,dBiases,1))
                    los+=loss(y,self.output_data[TRAINER])

                dWeights,dBiases=paraAdd(paraCopy(oldDWeights,oldDBiases,self.momentum),
                                         paraCopy(dWeights,dBiases,self.lr))

                self.__update(dWeights,dBiases)


                
                oldDWeights,oldDBiases=paraCopy(dWeights,dBiases,1)

                #print np.argmax(y),self.raw_class_labels[TRAINER]
                #if np.argmax(y)==self.raw_class_labels[TRAINER]:

                 #   accuracy+=1
                print 'Epoch : ',i,'Step : ',j,'loss : ',los
                #print 'acc=',accuracy*1.0/sets

    def fit(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.random.rand(self.layers[i+1],self.layers[i]))
            self.biases.append(np.random.rand(self.layers[i+1]))
            #print self.biases[-1].shape
        self.__train()
        
    def resume(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.load('./temp/weights_'+str(i)+'.npy'))
            self.biases.append(np.load('./temp/biases_'+str(i)+'.npy'))
            #print self.weights[-1].shape
        self.__train()
        
        
    def predict(self,X):
        return [np.argmax(self.__forward_propagation_test(i)) for i in X]    
        
    

In [561]:
model=NN_Model(hidden_layers=(128,128),lr=0.00005,max_iter=100,momentum=0.5,batch_size=25)
training_data=np.array(training_data)

In [ ]:
model.resume(training_data,training_data_class)


Epoch :  0 Step :  0 loss :  63.0829081337
Epoch :  0 Step :  1 loss :  56.586256349
Epoch :  0 Step :  2 loss :  39.9946238042
Epoch :  0 Step :  3 loss :  48.4443306802
Epoch :  0 Step :  4 loss :  39.5442671018
Epoch :  0 Step :  5 loss :  41.2252231366
Epoch :  0 Step :  6 loss :  69.9104201561
Epoch :  0 Step :  7 loss :  75.4728187456
Epoch :  0 Step :  8 loss :  47.6637544872
Epoch :  0 Step :  9 loss :  83.7019101602
Epoch :  0 Step :  10 loss :  53.1838401338
Epoch :  0 Step :  11 loss :  38.9499095139
Epoch :  0 Step :  12 loss :  67.7043572004
Epoch :  0 Step :  13 loss :  62.5538935449
Epoch :  0 Step :  14 loss :  34.4981948778
Epoch :  0 Step :  15 loss :  48.1357377841
Epoch :  0 Step :  16 loss :  49.500369245
Epoch :  0 Step :  17 loss :  61.9527185777
Epoch :  0 Step :  18 loss :  67.6163680463
Epoch :  0 Step :  19 loss :  38.3594126944
Epoch :  0 Step :  20 loss :  55.367826607
Epoch :  0 Step :  21 loss :  45.5514046951
Epoch :  0 Step :  22 loss :  52.5560788701
E

Epoch :  0 Step :  186 loss :  31.9574766094
Epoch :  0 Step :  187 loss :  64.9473539687
Epoch :  0 Step :  188 loss :  64.4343045123
Epoch :  0 Step :  189 loss :  44.4215215921
Epoch :  0 Step :  190 loss :  31.1785530146
Epoch :  0 Step :  191 loss :  54.7663826031
Epoch :  0 Step :  192 loss :  60.9186710975
Epoch :  0 Step :  193 loss :  46.3247954317
Epoch :  0 Step :  194 loss :  31.7675851794
Epoch :  0 Step :  195 loss :  38.0582619239
Epoch :  0 Step :  196 loss :  40.4338781067
Epoch :  0 Step :  197 loss :  40.1918297165
Epoch :  0 Step :  198 loss :  39.1133101386
Epoch :  0 Step :  199 loss :  47.8813513167
Epoch :  0 Step :  200 loss :  86.3101350123
Epoch :  0 Step :  201 loss :  61.6015643066
Epoch :  0 Step :  202 loss :  34.5888047926
Epoch :  0 Step :  203 loss :  59.4406406895
Epoch :  0 Step :  204 loss :  56.8162634042
Epoch :  0 Step :  205 loss :  93.6145723683
Epoch :  0 Step :  206 loss :  41.1221406173
Epoch :  0 Step :  207 loss :  57.8577411943
Epoch :  0

Epoch :  0 Step :  370 loss :  34.4810348385
Epoch :  0 Step :  371 loss :  59.091737286
Epoch :  0 Step :  372 loss :  56.7991994073
Epoch :  0 Step :  373 loss :  62.5651556698
Epoch :  0 Step :  374 loss :  80.886083507
Epoch :  0 Step :  375 loss :  38.3648156908
Epoch :  0 Step :  376 loss :  78.1615495592
Epoch :  0 Step :  377 loss :  90.7640176991
Epoch :  0 Step :  378 loss :  39.5516492384
Epoch :  0 Step :  379 loss :  41.2294826199
Epoch :  0 Step :  380 loss :  109.574493577
Epoch :  0 Step :  381 loss :  110.358228912
Epoch :  0 Step :  382 loss :  43.648430065
Epoch :  0 Step :  383 loss :  78.2187004408
Epoch :  0 Step :  384 loss :  78.9309643349
Epoch :  0 Step :  385 loss :  55.6465255059
Epoch :  0 Step :  386 loss :  52.6548958876
Epoch :  0 Step :  387 loss :  63.1991539556
Epoch :  0 Step :  388 loss :  54.2789337719
Epoch :  0 Step :  389 loss :  48.5802917158
Epoch :  0 Step :  390 loss :  98.6895502718
Epoch :  0 Step :  391 loss :  51.8246104969
Epoch :  0 St

Epoch :  0 Step :  556 loss :  47.1541791453
Epoch :  0 Step :  557 loss :  48.805717151
Epoch :  0 Step :  558 loss :  49.6573186189
Epoch :  0 Step :  559 loss :  54.3318791242
Epoch :  0 Step :  560 loss :  43.7468332564
Epoch :  0 Step :  561 loss :  54.2126366426
Epoch :  0 Step :  562 loss :  100.311738369
Epoch :  0 Step :  563 loss :  87.1694071527
Epoch :  0 Step :  564 loss :  107.117045284
Epoch :  0 Step :  565 loss :  40.9341859384
Epoch :  0 Step :  566 loss :  66.7410183251
Epoch :  0 Step :  567 loss :  51.8483136462
Epoch :  0 Step :  568 loss :  40.1653300634
Epoch :  0 Step :  569 loss :  52.6372364838
Epoch :  0 Step :  570 loss :  96.7752617669
Epoch :  0 Step :  571 loss :  84.3367503219
Epoch :  0 Step :  572 loss :  138.400889668
Epoch :  0 Step :  573 loss :  41.2587169659
Epoch :  0 Step :  574 loss :  60.7741277786
Epoch :  0 Step :  575 loss :  63.9312967506
Epoch :  0 Step :  576 loss :  33.6278237803
Epoch :  0 Step :  577 loss :  70.416975071
Epoch :  0 S

Epoch :  0 Step :  742 loss :  78.4229826641
Epoch :  0 Step :  743 loss :  46.3635956021
Epoch :  0 Step :  744 loss :  48.8020203021
Epoch :  0 Step :  745 loss :  40.1431974531
Epoch :  0 Step :  746 loss :  31.4760596151
Epoch :  0 Step :  747 loss :  76.9267392732
Epoch :  0 Step :  748 loss :  72.9145481322
Epoch :  0 Step :  749 loss :  72.903594434
Epoch :  0 Step :  750 loss :  63.3140462742
Epoch :  0 Step :  751 loss :  116.85056902
Epoch :  0 Step :  752 loss :  50.9733044203
Epoch :  0 Step :  753 loss :  56.0733210325
Epoch :  0 Step :  754 loss :  74.0272573118
Epoch :  0 Step :  755 loss :  101.744956788
Epoch :  0 Step :  756 loss :  56.8764344169
Epoch :  0 Step :  757 loss :  48.4483857748
Epoch :  0 Step :  758 loss :  60.3136368602
Epoch :  0 Step :  759 loss :  45.9391049494
Epoch :  0 Step :  760 loss :  79.6927673104
Epoch :  0 Step :  761 loss :  34.5808939614
Epoch :  0 Step :  762 loss :  68.4158837302
Epoch :  0 Step :  763 loss :  74.5540929974
Epoch :  0 S

...

Epoch :  2 Step :  538 loss :  67.1252562838
Epoch :  2 Step :  539 loss :  49.3215807907
Epoch :  2 Step :  540 loss :  74.4288387176
Epoch :  2 Step :  541 loss :  41.0301872327
Epoch :  2 Step :  542 loss :  57.234357715
Epoch :  2 Step :  543 loss :  126.331551256
Epoch :  2 Step :  544 loss :  88.1709335791
Epoch :  2 Step :  545 loss :  40.7543183015
Epoch :  2 Step :  546 loss :  60.9661563472
Epoch :  2 Step :  547 loss :  37.2121716132
Epoch :  2 Step :  548 loss :  67.7444123499
Epoch :  2 Step :  549 loss :  71.1317292062
Epoch :  2 Step :  550 loss :  60.8025458893
Epoch :  2 Step :  551 loss :  37.2287764362
Epoch :  2 Step :  552 loss :  69.8842898857
Epoch :  2 Step :  553 loss :  44.9758546167
Epoch :  2 Step :  554 loss :  33.4025769059
Epoch :  2 Step :  555 loss :  70.7431250238
Epoch :  2 Step :  556 loss :  46.5139606316
Epoch :  2 Step :  557 loss :  48.2879150342
Epoch :  2 Step :  558 loss :  49.0283328695
Epoch :  2 Step :  559 loss :  53.5662662715
Epoch :  2 

Epoch :  2 Step :  725 loss :  36.1406843866
Epoch :  2 Step :  726 loss :  40.8696797433
Epoch :  2 Step :  727 loss :  68.2573520797
Epoch :  2 Step :  728 loss :  39.517167555
Epoch :  2 Step :  729 loss :  49.5117028129
Epoch :  2 Step :  730 loss :  48.1641733719
Epoch :  2 Step :  731 loss :  66.3348479069
Epoch :  2 Step :  732 loss :  76.6172579153
Epoch :  2 Step :  733 loss :  44.8374175319
Epoch :  2 Step :  734 loss :  65.9782334305
Epoch :  2 Step :  735 loss :  51.2963197689
Epoch :  2 Step :  736 loss :  70.4552028894
Epoch :  2 Step :  737 loss :  50.3283970037
Epoch :  2 Step :  738 loss :  74.3641601568
Epoch :  2 Step :  739 loss :  68.3341468319
Epoch :  2 Step :  740 loss :  31.1073635232
Epoch :  2 Step :  741 loss :  108.667307041
Epoch :  2 Step :  742 loss :  75.0987913147
Epoch :  2 Step :  743 loss :  46.8719276534
Epoch :  2 Step :  744 loss :  47.2400936436
Epoch :  2 Step :  745 loss :  39.5602074623
Epoch :  2 Step :  746 loss :  31.9099299754
Epoch :  2 

Epoch :  2 Step :  911 loss :  75.9002934564
Epoch :  2 Step :  912 loss :  60.9835407725
Epoch :  2 Step :  913 loss :  62.7427812405
Epoch :  2 Step :  914 loss :  36.4220215695
Epoch :  2 Step :  915 loss :  63.5872956928
Epoch :  2 Step :  916 loss :  33.751373342
Epoch :  2 Step :  917 loss :  71.7503053257
Epoch :  2 Step :  918 loss :  53.2240391805
Epoch :  2 Step :  919 loss :  54.3621343964
Epoch :  2 Step :  920 loss :  55.4006700614
Epoch :  2 Step :  921 loss :  73.1383593628
Epoch :  2 Step :  922 loss :  54.5501548011
Epoch :  2 Step :  923 loss :  31.4513073723
Epoch :  2 Step :  924 loss :  46.6137973004
Epoch :  2 Step :  925 loss :  49.1957243861
Epoch :  2 Step :  926 loss :  66.0329171199
Epoch :  2 Step :  927 loss :  39.8632837066
Epoch :  2 Step :  928 loss :  48.834857203
Epoch :  2 Step :  929 loss :  46.6849556854
Epoch :  2 Step :  930 loss :  58.8757195089
Epoch :  2 Step :  931 loss :  52.7282049787
Epoch :  2 Step :  932 loss :  88.5473100571
Epoch :  2 S

Epoch :  2 Step :  1095 loss :  52.1961435302
Epoch :  2 Step :  1096 loss :  92.8715288484
Epoch :  2 Step :  1097 loss :  58.0444617673
Epoch :  2 Step :  1098 loss :  54.4876846851
Epoch :  2 Step :  1099 loss :  67.1284317703
Epoch :  2 Step :  1100 loss :  76.6881867031
Epoch :  2 Step :  1101 loss :  39.8739103037
Epoch :  2 Step :  1102 loss :  83.2454926096
Epoch :  2 Step :  1103 loss :  87.137074257
Epoch :  2 Step :  1104 loss :  74.2760627653
Epoch :  2 Step :  1105 loss :  50.1413132666
Epoch :  2 Step :  1106 loss :  58.322200727
Epoch :  2 Step :  1107 loss :  78.2954938739
Epoch :  2 Step :  1108 loss :  57.7395112737
Epoch :  2 Step :  1109 loss :  59.2903568999
Epoch :  2 Step :  1110 loss :  46.8376248422
Epoch :  2 Step :  1111 loss :  37.5458321856
Epoch :  2 Step :  1112 loss :  103.512447268
Epoch :  2 Step :  1113 loss :  46.4967377606
Epoch :  2 Step :  1114 loss :  46.9400464247
Epoch :  2 Step :  1115 loss :  39.3470552431
Epoch :  2 Step :  1116 loss :  49.1

Epoch :  2 Step :  1278 loss :  32.4289691479
Epoch :  2 Step :  1279 loss :  37.9463814501
Epoch :  2 Step :  1280 loss :  61.9723872562
Epoch :  2 Step :  1281 loss :  48.7127455393
Epoch :  2 Step :  1282 loss :  87.2126010438
Epoch :  2 Step :  1283 loss :  54.223312852
Epoch :  2 Step :  1284 loss :  54.5013800895
Epoch :  2 Step :  1285 loss :  73.4152956826
Epoch :  2 Step :  1286 loss :  68.1092849984
Epoch :  2 Step :  1287 loss :  54.1566006216
Epoch :  2 Step :  1288 loss :  52.071021667
Epoch :  2 Step :  1289 loss :  69.5884100393
Epoch :  2 Step :  1290 loss :  50.4152023792
Epoch :  2 Step :  1291 loss :  79.9757965238
Epoch :  2 Step :  1292 loss :  37.0908322516
Epoch :  2 Step :  1293 loss :  62.1188919072
Epoch :  2 Step :  1294 loss :  44.5977359969
Epoch :  2 Step :  1295 loss :  49.6474884125
Epoch :  2 Step :  1296 loss :  96.2141286651
Epoch :  2 Step :  1297 loss :  52.8322788769
Epoch :  2 Step :  1298 loss :  51.1463348854
Epoch :  2 Step :  1299 loss :  59.3

In [559]:
len(validating_data)-np.count_nonzero(model.predict(validating_data)-validating_data_class)

2376

In [560]:
dumpModel(model)

In [521]:
testing_data=sk.preprocessing.normalize(test.iloc[:,1:785].copy().as_matrix())
testing_data=clf.transform(testing_data)

In [524]:
test_results=model.predict(testing_data)

In [526]:
tmp=[i for i in range(10000)]

In [530]:
csv_result=np.array(zip(tmp,test_results))

In [531]:
print csv_result

[[   0    4]
 [   1    4]
 [   2    2]
 ..., 
 [9997    3]
 [9998    7]
 [9999    6]]


In [533]:
np.savetxt('./res.csv',csv_result,delimiter=',',fmt='%d')